In [1]:
import datasets

data = datasets.load_dataset('gmongaras/book_BERT_512')

/Users/Yourui/Documents/bloom-filters/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import contextlib

with open('tokens.txt', 'w') as f:
    for i, row in enumerate(data['train']):
        tokens = row['input_ids']
        with contextlib.suppress(ValueError):
            tokens = tokens[:tokens.index(0)]
        for sequence in [tokens[i:i+4] for i in range(len(tokens)-3)]:
            f.write(str(sequence) + '\n')
        
        if i % 1000 == 0:
            print(f'{i}th row processed')

0th row processed
1000th row processed
2000th row processed
3000th row processed
4000th row processed
5000th row processed
6000th row processed
7000th row processed
8000th row processed
9000th row processed
10000th row processed
11000th row processed
12000th row processed
13000th row processed
14000th row processed
15000th row processed
16000th row processed
17000th row processed
18000th row processed
19000th row processed
